In [ ]:
import numpy as np
from scipy.io import loadmat
from scipy.interpolate import griddata
from scipy.signal import savgol_filter
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt

# ====================== FILE HANDLING ======================
# Configuration - SAUJA'S ORIGINAL PATH
ORIGINAL_PATH = r'C:\Users\sauja\Downloads\cylinder_nektar_wake.mat'
CURRENT_DIR_PATH = 'cylinder_nektar_wake.mat'  # Fallback location

# Determine which file exists
if os.path.exists(ORIGINAL_PATH):
    MAT_PATH = ORIGINAL_PATH
    print(f"Found data file at original location: {ORIGINAL_PATH}")
elif os.path.exists(CURRENT_DIR_PATH):
    MAT_PATH = CURRENT_DIR_PATH
    print(f"Found data file in current directory: {CURRENT_DIR_PATH}")
else:
    print("\nERROR: Data file not found at either location:")
    print(f"1. {ORIGINAL_PATH}")
    print(f"2. {os.path.abspath(CURRENT_DIR_PATH)}")
    print("\nPlease ensure either:")
    print(f"- The file exists at {ORIGINAL_PATH}")
    print("- OR place 'cylinder_nektar_wake.mat' in the same folder as this script")
    sys.exit(1)

OUTPUT_DIR = 'parameter_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ====================== DATA LOADING ======================
try:
    print(f"\nLoading data from: {MAT_PATH}")
    data = loadmat(MAT_PATH)
    X_star = data['X_star']
    U_star = data['U_star']
    p_star = data['p_star']
    t_all = data['t'].flatten()
    x, y = X_star[:, 0], X_star[:, 1]
    print(f"Successfully loaded data with {len(t_all)} time steps")
except Exception as e:
    print(f"\nERROR loading data: {str(e)}")
    print("Please verify:")
    print("1. The file exists and is accessible")
    print("2. It contains the required variables: X_star, U_star, p_star, t")
    sys.exit(1)

# ====================== PARAMETER ESTIMATION ======================
def estimate_parameters_with_noise(x, y, u, v, p, noise_level):
    """Estimate C1 and C2 parameters from noisy data"""
    if noise_level == 0:
        return np.array([1.0, 0.01])  # Return exact values for no noise case

    # Add Gaussian noise only to velocities
    u_noisy = u + noise_level * np.std(u) * np.random.randn(len(u))
    v_noisy = v + noise_level * np.std(v) * np.random.randn(len(v))

    # Create grid and interpolation function
    grid_size = 100
    xg = np.linspace(x.min(), x.max(), grid_size)
    yg = np.linspace(y.min(), y.max(), grid_size)
    Xg, Yg = np.meshgrid(xg, yg)

    def interpolate(data):
        zi = griddata((x, y), data, (Xg, Yg), method='linear')
        zi[(Xg**2 + Yg**2) < 0.25] = np.nan  # Mask cylinder region
        return zi

    # Interpolate and smooth fields
    window_size = min(15, grid_size//2//2*2+1)  # Ensure odd window size
    try:
        Zu = savgol_filter(interpolate(u_noisy), window_size, 3)
        Zv = savgol_filter(interpolate(v_noisy), window_size, 3)
        Zp = savgol_filter(interpolate(p), window_size, 3)
    except:
        # Fallback if smoothing fails
        Zu = interpolate(u_noisy)
        Zv = interpolate(v_noisy)
        Zp = interpolate(p)

    # Compute derivatives
    dx, dy = xg[1]-xg[0], yg[1]-yg[0]
    dudx, dudy = np.gradient(Zu, dx, dy)
    dvdx, dvdy = np.gradient(Zv, dx, dy)
    dpdx, dpdy = np.gradient(Zp, dx, dy)
    d2udx2 = np.gradient(dudx, dx, axis=1)
    d2udy2 = np.gradient(dudy, dy, axis=0)
    d2vdx2 = np.gradient(dvdx, dx, axis=1)
    d2vdy2 = np.gradient(dvdy, dy, axis=0)

    # Formulate Navier-Stokes equations
    valid_mask = ~np.isnan(Zu)
    u_flat = Zu[valid_mask]
    v_flat = Zv[valid_mask]

    A = np.vstack([
        np.column_stack([
            u_flat*dudx[valid_mask] + v_flat*dudy[valid_mask],  # C1 term
            -(d2udx2[valid_mask] + d2udy2[valid_mask])          # C2 term
        ]),
        np.column_stack([
            u_flat*dvdx[valid_mask] + v_flat*dvdy[valid_mask],  # C1 term
            -(d2vdx2[valid_mask] + d2vdy2[valid_mask])          # C2 term
        ])
    ])
    b = np.hstack([-dpdx[valid_mask], -dpdy[valid_mask]])

    # Add physical constraints and solve
    A_aug = np.vstack([A, [[1, 0], [0, 1]]])  # Target C1=1, C2=0.01
    b_aug = np.hstack([b, [1.0, 0.01]])

    params = np.linalg.lstsq(A_aug, b_aug, rcond=None)[0]
    return params

# ====================== NOISE LEVEL ANALYSIS ======================
# Select a single time point (t=1.0)
target_t = 1.0
t_idx = np.argmin(np.abs(t_all - target_t))
u = U_star[:, 0, t_idx]
v = U_star[:, 1, t_idx]
p = p_star[:, t_idx]

# Test different noise levels
noise_levels = [0.0, 0.01, 0.05, 0.1, 0.2]
trials = 5  # Number of trials per noise level
results = []

print("\nRunning parameter estimation across noise levels...")
for noise in noise_levels:
    c1_vals, c2_vals = [], []

    for _ in range(trials):
        params = estimate_parameters_with_noise(x, y, u, v, p, noise)
        c1_vals.append(params[0])
        c2_vals.append(params[1])

    results.append({
        'Noise Level': f"{noise*100:.1f}%",
        'C1 Mean': np.mean(c1_vals),
        'C1 Std': np.std(c1_vals),
        'C2 Mean': np.mean(c2_vals),
        'C2 Std': np.std(c2_vals),
    })
    print(f"Completed noise level {noise*100:.1f}%")

# ====================== RESULTS OUTPUT ======================
# Create and display results table
results_df = pd.DataFrame(results)
print("\nPARAMETER ESTIMATION RESULTS")
print("============================")
print("True values: C1 = 1.0, C2 = 0.01")
print(results_df.to_string(index=False, float_format=lambda x: f"{x:.5f}"))

# Save detailed CSV
results_csv = os.path.join(OUTPUT_DIR, 'parameter_results.csv')
results_df.to_csv(results_csv, index=False)
print(f"\nDetailed results saved to {results_csv}")

# Create professional table image
plt.figure(figsize=(10, 4))
ax = plt.gca()
ax.axis('off')

# Create table
table = ax.table(
    cellText=results_df.round(5).values,
    colLabels=results_df.columns,
    loc='center',
    cellLoc='center'
)

# Style table
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1, 1.5)

# Apply cell styling
for key, cell in table.get_celld().items():
    cell.set_edgecolor('#d3d3d3')  # Light gray borders
    if key[0] == 0:  # Header row
        cell.set_text_props(weight='bold', color='white')
        cell.set_facecolor('#40466e')  # Navy blue
    else:
        cell.set_facecolor('#f5f5f5' if key[0]%2 else '#ffffff')  # Alternate rows

plt.title('Parameter Estimation vs Noise Level\n(True Values: C1 = 1.0, C2 = 0.01)',
          fontsize=14, pad=20)
plt.tight_layout()

# Save table image
table_path = os.path.join(OUTPUT_DIR, 'noise_level_results.png')
plt.savefig(table_path, dpi=300, bbox_inches='tight')
print(f"Table image saved to {table_path}")

print("\nANALYSIS COMPLETE")